In [2]:
!pip install imblearn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
data = pd.read_csv("churn.csv")


In [ ]:

data_cleaned = data.drop(columns=["RowNumber", "CustomerId", "Surname"])


In [ ]:
categorical_columns = ["Geography", "Gender"]
numerical_columns = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary"]
target_column = "Exited"


In [ ]:
numerical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="first"))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numerical_pipeline, numerical_columns),
    ("cat", categorical_pipeline, categorical_columns)
])


In [ ]:

X = data_cleaned.drop(columns=[target_column])
y = data_cleaned[target_column]

X_preprocessed = preprocessor.fit_transform(X)

In [ ]:

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_preprocessed, y)

In [ ]:

onehot_columns = preprocessor.named_transformers_["cat"].named_steps["onehot"].get_feature_names_out(categorical_columns)
processed_columns = list(numerical_columns) + list(onehot_columns)

X_smote_df = pd.DataFrame(X_smote, columns=processed_columns)
final_dataset = pd.concat([X_smote_df, y_smote.reset_index(drop=True)], axis=1)

In [ ]:
final_dataset.to_csv("preprocessed_with_smote.csv", index=False)